In [ ]:
import import_ipynb
import numpy as np
import glob
import matplotlib.pyplot as plt
import datetime

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters
Test for optimal speed.

In [ ]:
batch_size = 32

## Model callbacks

In [ ]:
tb_log_dir = "../log/tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
cp_filepath = '../log/cps/{model.name}_cp_{epoch}.h5'

callbacks = [
    keras.callbacks.ModelCheckpoint(                        
                        filepath=cp_filepath, 

                        # 1 Displays messages when the callback takes an actio
                        verbose=1, 

                        save_weights_only=False,
                        save_freq=5*batch_size), # Checkpoint is saved every five epochs, checkpoints are missing?
                        tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)
]

## Data parameters

In [ ]:
image_height = 224
image_width = 224

## Creating the training dataset

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",

    # For two classes; face, noFace; for more classes multiclass
    label_mode='binary',

    color_mode='rgb',

    # To avoid default value
    batch_size=batch_size,

    image_size=(image_height, image_width),

    # Mixxing of the matrix
    shuffle=True,

    # "seed" argument for shuffling and validation of the data to make sure that there is no overlap between the training and validation subset.
    seed=123,

    validation_split=0.2,
    subset='training',

    # To make the attribut for changes visible in the code
    interpolation='bilinear',

    follow_links=False,

    # Missing dont create an error. Could be important for resizing. No error in both cases with or without a picture with different format and pixels.
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

## Creating the validation dataset

In [ ]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    # To avoid default value
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

## Creating the test dataset

In [ ]:
test_ds = []

## Load the base model

In [ ]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50(
                include_top=False,
                weights='imagenet',  # Load weights pre-trained on ImageNet.
                input_shape=(image_height, image_width, 3)
                )
base_model.trainable = False

## Creating a new model

In [ ]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

## Training the new model with new data
Amount of epochs is for the number of around 25.000 images very high. Because specially the non face images are very diverse.

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=10, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

## View Results

## Predict new images

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/000001.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))